# Working with REMO domains

The domain module should give some tools to work with preconfigured or user defined domains. Domains are defined as xarray datasets that will contain dimensions and coodinates according to CF-conventions. The pyremo domain module actually uses the [py-cordex](https://py-cordex.readthedocs.io/en/latest/domains.html) domains module in the background just with another set of tables.

**NOTE**: Please be aware that a remo domain is usually a little larger than the “official” cordex domain according to the [archive specification](https://is-enes-data.github.io/cordex_archive_specifications.pdf) since the regional model usually accounts for a “buffer” zone where the lateral boundary conditions are nudged.

## Working with domain information

In [ ]:
import pyremo as pr

The domain module contains some useful functions to work with cordex meta data, e.g., you can get some domain grid information using

In [ ]:
pr.domain_info("EUR-11")

All available cordex domains are in this table:

In [ ]:
pr.domains.table

## `EUR-11` example

The heart of the module are some functions that create a dataset from the grid information, e.g.

In [ ]:
eur11 = pr.remo_domain("EUR-11", dummy="topo")
eur11

The `dummy='topo'` argument means, we want a dummy variable in the dataset to see how the domain looks like. For the dummy topography, we use the `cdo topo` operator in the background. So maybe you have to install `python-cdo`, e.g., `conda install -c conda-forge python-cdo`. Working with xarray datasets means, that we can use all the nice functions of xarray including plotting, e.g.,

In [ ]:
eur11.topo.plot(cmap="terrain")

In [ ]:
eur11.topo.plot(x="lon", y="lat", cmap="terrain")

Let's define a slightly more sophisticated plotting function that uses cartopy for the right [projection](https://scitools.org.uk/cartopy/docs/latest/tutorials/understanding_transform.html) with a rotated pole:

In [ ]:
def plot(da, pole, vmin=None, vmax=None, borders=True, title=""):
    """plot a domain using the right projection with cartopy"""
    %matplotlib inline
    import cartopy.crs as ccrs
    import cartopy.feature as cf
    import matplotlib.pyplot as plt

    plt.figure(figsize=(20, 10))
    projection = ccrs.PlateCarree()
    transform = ccrs.RotatedPole(pole_latitude=pole[1], pole_longitude=pole[0])
    # ax = plt.axes(projection=projection)
    ax = plt.axes(projection=transform)
    # ax.set_extent([ds_sub.rlon.min(), ds_sub.rlon.max(), ds_sub.rlat.min(), ds_sub.rlat.max()], crs=transform)
    ax.gridlines(
        draw_labels=True,
        linewidth=0.5,
        color="gray",
        xlocs=range(-180, 180, 10),
        ylocs=range(-90, 90, 5),
    )
    da.plot(
        ax=ax,
        cmap="terrain",
        transform=transform,
        vmin=vmin,
        vmax=vmax,
        x="rlon",
        y="rlat",
    )
    ax.coastlines(resolution="50m", color="black", linewidth=1)
    if borders:
        ax.add_feature(cf.BORDERS)
    ax.set_title("")

In [ ]:
pole_eur = (
    eur11.rotated_latitude_longitude.grid_north_pole_longitude,
    eur11.rotated_latitude_longitude.grid_north_pole_latitude,
)
pole_eur

In [ ]:
plot(eur11.topo, pole_eur)

## User defined domain

The domains are created using the `create_dataset` function from the [`py-cordex`](https://py-cordex.readthedocs.io) package, e.g.:

In [ ]:
from cordex import create_dataset

Let's create the EUR-11 domain manually from the numbers in the table:

In [ ]:
eur11_user = create_dataset(
    nlon=433,
    nlat=433,
    dlon=0.11,
    dlat=0.11,
    ll_lon=-28.925,
    ll_lat=-23.925,
    pollon=-162.00,
    pollat=39.25,
    dummy="topo",
)

We can check that this gives the same result as our preconfigured domain.

In [ ]:
eur11_user.equals(eur11)

You can now use the `create_dataset` function to create any domain as an xarray dataset.

In [ ]:
afr11 = pr.remo_domain("AFR-11", dummy="topo")
afr11

In [ ]:
pole_afr = (
    afr11.rotated_latitude_longitude.grid_north_pole_longitude,
    afr11.rotated_latitude_longitude.grid_north_pole_latitude,
)
pole_afr

In [ ]:
plot(afr11.topo, pole_afr)

## Cropping the REMO domain

Sometimes it might be neccessary to crop the REMO data to the official CORDEX grid size, e.g., for cmorization. This can now easily be done like this:

In [ ]:
import numpy as np
from cordex import cordex_domain

eur11_cordex = cordex_domain("EUR-11", dummy="topo")

In [ ]:
# crop = eur11.sel(rlon=slice(eur11_cordex.rlon.min(), eur11_cordex.rlon.max()), rlat=slice(eur11_cordex.rlat.min(), eur11_cordex.rlat.max()))
crop = eur11.sel(rlon=eur11_cordex.rlon, rlat=eur11_cordex.rlat, method="nearest")

In [ ]:
plot(crop.topo, pole_eur)

In [ ]:
crop